<a href="https://colab.research.google.com/github/wakristensen/machine-learning-workshop/blob/main/01_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 EDA, Rens og kundefeatures

<div class="badges">
  <a>⏱️ 40 min</a>
  <a>👩‍💻 Hands-on</a>
  <a>📦 UK Online Retail 2010–2011</a>
</div>

I denne notebooken laster vi inn data, utforsker datastruktur, rydder verdier og lager kundesentrerte variabler som grunnlag for segmentering. Kjør cellene i rekkefølge

## Datasettet

Vi skal bruker et transaksjonsdatasett fra en britisk netthandel. Datasettet inneholder transaksjoner mellom 2010 og 2011.

Clustering kan brukes for å øke effektiviteten i markedsføringsstrategier og forbedre salget gjennom kundesegmentering og for analyseformål til innsikt, samt som variabler til ulike prediksjonsmodeller.

Vi ønsker å transformere transaksjonsdataene til et kundesentrisk datasett ved å skape nye kundesentriske variabler som legger til rette for å segmentere kunder i ulike grupper ved hjelp av clusteringsalgoritmer. K-means clusteirng skal vi bruke i denne notebooken.

Segmenteringen gir oss innsikt i de ulike profilenes særtrekk og preferanser.

Disse kan brukes til å f.eks. utvikle et anbefalingssystem som foreslår bestselgende produkter til kunder innenfor hvert segment. Det overordnede forretningsmålet knyttet til segmentering kan være for å styrke markedsføringens effektivitet og bidra til økt salg.

## Mål med denne delen
- Datavask og transformasjon, håndtere manglende verdier, duplikater og avvik
- Feature engineering, gjøre data kundesentrisk

## Innholdsfortegnelse

- **Steg 1 | Oppsett og initialisering**  
  - Steg 1.1 | Importere nødvendige biblioteker  
  - Steg 1.2 | Laste inn datasettet  

- **Steg 2 | Innledende dataanalyse**  
  - Steg 2.1 | Oversikt over datasettet  
  - Steg 2.2 | Oppsummerende statistikk  

- **Steg 3 | Datavask og transformasjon**  
  - Steg 3.1 | Håndtering av manglende verdier  
  - Steg 3.2 | Håndtering av duplikater  
  - Steg 3.3 | Behandling av kansellerte transaksjoner  
  - Steg 3.4 | Rette opp StockCode-anomalier  
  - Steg 3.5 | Rense beskrivelseskolonnen  
  - Steg 3.6 | Behandle nullverdier i UnitPrice

## Steg 1 · Oppsett og initialisering


### 1.1 | Importere nødvendige biblioteker


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1.2 | Laste inn datasettet


In [ ]:
file_id = "1Ku-y5BE5CdGQwzEMQ491cfIYWILS6T8U"
url = f"https://drive.google.com/uc?id={file_id}"

df = pd.read_csv(url, encoding="latin1")
print("Data lastet inn")
print("")
display(df.head(3))
print("")

#### Beskrivelse av datasettet

|  Kolonne        | Beskrivelse |
|-----------------|-------------|
| **InvoiceNo**   | Id som representerer hver unike transaksjon. Hvis koden starter med bokstaven **C**, betyr det at transaksjonen er en kansellering. |
| **StockCode**   | Unik ID per produkt. |
| **Description** | Tekstbeskrivelse av hvert produkt.|
| **Quantity**    | Antall enheter av et produkt i en transaksjon.|
| **InvoiceDate** | Dato og klokkeslett for transaksjonen. |
| **UnitPrice**   | Enhetspris for produktet (i pund).|
| **CustomerID**  | Unik id per kunde. |
| **Country**     | Landet kunden tilhører / bestiller til |


## Steg 2 · Dataoversikt
Hva finnes i datasettet, hvilke typer, hvor mye mangler.


### 2.1 | Oversikt over datasettet

In [ ]:
df.head(10)

#### Oppgave – Utforske datastruktur og datatyper

Kjør `df.info()` for å undersøke datasettet.

1. Hvor mange rader og kolonner finnes i datasettet?  
2. Hvilke datatyper har de ulike kolonnene?  
3. Hvilke kolonner har manglende verdier?
4. Hvilken datatype er **InvoiceDate**, hva burde den være?  
5. Hvilken datatype er **CustomerID**, og hva forteller de mange `NaN`-verdiene oss?


In [ ]:
df.info()

<details>
<summary>📖 Svarforslag (klikk for å åpne)</summary>

1. **Rader og kolonner:** 541 909 rader og 8 kolonner.  
2. **Datatyper:**  
   - InvoiceNo → object (streng)  
   - StockCode → object (streng)  
   - Description → object (streng, med NaN)  
   - Quantity → int64  
   - InvoiceDate → object (streng, men burde være datetime)  
   - UnitPrice → float64  
   - CustomerID → float64 (egentlig en ID, ikke et tall, når vi har håndternt null-verdier kan vi konvertere den til *`Int64`* eller *`string`*.
   - Country → object (streng)  

3. **Manglende verdier:**  
   - Description har 1 454 manglende verdier  
   - CustomerID har 135 080 manglende verdier  

4. **InvoiceDate:** Er lagret som `object` (streng), men bør konverteres til `datetime` med `pd.to_datetime()`.  

5. **CustomerID:** Er `float64` fordi kolonnen har `NaN`. Når vi håndterer NaN kan vi gjøre den til heltall (`Int64`).  

6. **object:** I Pandas betyr dette vanligvis **tekst** (string), men kan i prinsippet være hva som helst. I praksis = “ikke-numerisk data”.  

</details>


###  2.2 | Statistikk om datasettet

Vi lager oppsummerende statistikk for å få innsikt i fordelingen av dataene:


In [ ]:
# Oppsummering for numeriske variabler
df.describe()

In [ ]:
# Summary statistics for categorical variables
df.describe(include='object').T

#### Oppgave - statistikk om datasettet
Se tabellene over

1. Snitt antall produkter per transaksjon?  
2. Hvorfor er det negative verdier i Quantity?  
3. Hva er snitt enhetspris (UnitPrice), og hvorfor kan negative priser være problematiske?  
4. Hvor mange kunder (CustomerID) finnes det, og hvor mange verdier mangler?  
6. Hvilket produkt (StockCode og Description) forekommer oftest?  
7. Hvor mange land er representert, og hvilket land dominerer?  

<details>
<summary>📖 Svarforslag (klikk for å åpne)</summary>

- **Quantity:**  
  - Gjennomsnitt ≈ 9,55 per transaksjon.  
  - Verdiene varierer fra -80 995 til 80 995.  
  - Negative verdier = returer eller kanselleringer.  
  - Stor spredning (høyt standardavvik) og tydelige uteliggere.  

- **UnitPrice:**  
  - Gjennomsnitt ≈ 4,61.  
  - Verdiene varierer fra -11 062,06 til 38 970.  
  - Negative priser gir ikke mening → tyder på feil i data.  
  - Stor forskjell mellom maks og 75%-percentilen → uteliggere.  

- **CustomerID:**  
  - 406 829 ikke-null verdier (mange mangler).  
  - ID-er fra 12 346 til 18 287 → representerer unike kunder.  

- **InvoiceNo:**  
  - Ca. 25 900 unike fakturaer → 25 900 transaksjoner.  
  - Mest frekvente: **573585**, dukker opp 1114 ganger.  

- **StockCode:**  
  - 4 070 unike produktkoder.  
  - Mest frekvente: **85123A**, dukker opp 2313 ganger.  

- **Description:**  
  - 4 223 unike produktbeskrivelser.  
  - Mest frekvente: **“WHITE HANGING HEART T-LIGHT HOLDER”**, dukker opp 2369 ganger.  
  - Har manglende verdier.
  - Enkelte beskrivelser forekommer flere ganger på ulike *stockCodes*.

- **Country:**  
  - 38 land representert.  
  - Storbritannia dominerer (≈ 91,4 % av transaksjonene).  

</details>


## Steg 3 · Datavask og transformasjon
I dette steget renser og transformerer vi datasettet for å gjøre det mer presist og brukbart.

Det inkluderer:
- Håndtere manglende verdier
- Fjerne duplikater
- rette opp i rare produktkoder og beskrivelser
- Andre justeringer for å forberede dataene til videre analyse og modellering.  


### 3.1 | Håndtering av manglende verdier
Vi finner prosentandelen med manglende verdier i hver kolonne, og velger den mest hensiktsmessige strategien for å håndtere dem:








In [ ]:
# prosentandel av manglende verdier for hver kolonne
missing_data = df.isnull().sum()
missing_percentage = (missing_data[missing_data > 0] / df.shape[0]) * 100

# Sorterer etter flest manglende verdier
missing_percentage.sort_values(ascending=True, inplace=True)

# Plotter
fig, ax = plt.subplots(figsize=(15, 4))
ax.barh(missing_percentage.index, missing_percentage, color='#ff6200')

# Legger til prosenter
for i, (value, name) in enumerate(zip(missing_percentage, missing_percentage.index)):
    ax.text(value+0.5, i, f"{value:.2f}%", ha='left', va='center', fontweight='bold', fontsize=18, color='black')

# Set x-axis limit
ax.set_xlim([0, 40])

# Add title and xlabel
plt.title("Percentage of Missing Values", fontweight='bold', fontsize=22)
plt.xlabel('Percentages (%)', fontsize=16)
plt.show()

In [ ]:
# Ser på rader som mangler verdier i 'CustomerID' eller 'Description'
df[df['CustomerID'].isnull() | df['Description'].isnull()]

#### Oppgave – Strategi for manglende verdier

Med oversikt over manglende verdier vurder:

- Hvilke strategier kan vi gjøre for å håndtere dette (Fjerne rader, fylle inn med verdier, ignorere)?

<details>
<summary> Forslag til svar </summary>

**CustomerID (24,93 % manglende verdier)**  

- Denne kolonnen er sentral for å kunne klustre kunder og bygge et anbefalingssystem.
- Å fylle inn så stor andel manglende verdier vil fort gi skjevheter eller støy.
- Siden klustringen skal baseres på kundeadferd og preferanser, er det viktig å ha nøyaktige data på kundeidentifikatorene.
- Beste løsning: Fjerne rader som mangler *CustomerID*, for å sikre kvalitet i klustrene og analysen.  

---

**Description (0,27 % manglende verdier)**  

- Liten andel manglende verdier.
- Inkonsistens i dataene, samme *StockCode* har ikke alltid har samme *Description*.
- På grunn av disse inkonsistensene vil det være ikke så pålitelig å fylle inn de manglende verdiene basert på *StockCode*.
- Beste løsning: Siden andelen manglende verdier er så lav, er det fornuftig å fjerne radene med manglende *Description*, slik at vi unngår å spre feil og inkonsistens i analysene.

---

Ved å fjerne radene med manglende verdier i *CustomerID* og *Description* får vi et renere, mer pålitelig og bedre datasett, det er viktig for god klustring og bygge et bra anbefalingssystem.


In [ ]:
# Fjerner rader med null-verdier i 'CustomerID' og 'Description' kolonnene
df = df.dropna(subset=['CustomerID', 'Description'])

In [ ]:
# Verifiserer at manglende verdier er fjernet
df.isnull().sum()

### 3.2 | Håndtering av duplikater
Neste steg er å finne dupliserte rader i datasettet

In [ ]:
# Finne dupliserte rader
duplicate_rows = df[df.duplicated(keep=False)]

# Sortere datafor å se dupliserte rader ved siden av hverandre
duplicate_rows_sorted = duplicate_rows.sort_values(by=['InvoiceNo', 'StockCode', 'Description', 'CustomerID', 'Quantity'])

# Inspiserer dupliserte rader
duplicate_rows_sorted

#### Strategi for håndtering av duplikater

1. Hvor mange identiske rader som finnes i datasettet.  
2. Hva kan være årsaken til at vi har helt identiske rader?
3. Hva kan konsekvensen bli dersom vi beholder duplikatene i datasettet når vi senere skal klustre kunder eller lage et anbefalingssystem?  


<details>
<summary> Forslag til svar </summary>

- **Antall duplikater:** Det finnes rader som er 100 % identiske, også med samme tidspunkt.  
- **Årsak:** Mest sannsynlig feil i dataregistreringen, ikke reelle gjentatte kjøp.  
- **Strategi:**  
  - Hvis vi beholder dem, kan det introdusere støy og feil i analysene.  
  - Ved å fjerne duplikater får vi et renere datasett.  

**Konklusjon:**  
Vi fjerner de identiske duplikat-radene for å:  
- Sikre at klynger reflekterer unike kjøpsmønstre per kunde.  
- Vil bidra til å lage et mer presist anbefalingssystem, basert på produkter som faktisk kjøpes mest.
</details>


In [ ]:
# Vise antall dupliserte rader
print(f"Datasettet inneholder {df.duplicated().sum()} dupliserte rader som bør fjernes.")

# Fjerne dupliserte rader
df.drop_duplicates(inplace=True)

# Sjekke at de er fjernet
print(f"Etter fjerning: {df.duplicated().sum()} dupliserte rader igjen.")
print(f"Antall rader i datasettet nå: {len(df)}")

### 3.3 | Behandling av kansellerte transaksjoner
For å forstå kundeadferd og preferanser må vi ta hensyn til transaksjoner som ble kansellert.  

Først identifiserer vi transaksjonene ved å filtrere radene der *InvoiceNo* starter med "C".

Deretter analyserer vi disse radene for å se om vi finner noen felles kjennetegn eller mønstre for disse.  


1. Vi lager  en ny kolonne `Transaction_Status` som markerer en rad som `Cancelled` hvis `InvoiceNo` starter med "C", ellers `Completed`.
2. Hvor mange kansellerte transaksjoner er det i datasettet?  
3. Vi analyser kansellerte rader med `.describe()` for å se om de har spesielle mønstre (f.eks. i `Quantity` eller `UnitPrice`).  

In [ ]:
# Lager Transaction_Status-kolonnen
df['Transaction_Status'] = np.where(df['InvoiceNo'].astype(str).str.startswith('C'),
                                    'Cancelled', 'Completed')
# Filtrer kansellerte transaksjoner og lagrer som egen variabel
cancelled_transactions = df[df['Transaction_Status'] == 'Cancelled']

# .describe() på radene
cancelled_transactions.describe().drop('CustomerID', axis=1)


#### Observasjoner fra kansellerte transaksjoner

- Alle verdiene i **Quantity** for de kansellerte transaksjonene er negative. Det bekrefter at radene representerer kansellerte eller returnerte ordre.  
- **UnitPrice**-kolonnen har stor variasjon, noe som viser at et bredt spekter av produkter – inngår i de kansellerte transaksjonene.  


#### Strategi for håndtering av kansellerte transaksjoner

Med tanke på målet om å klustre kunder basert på kjøpsadferd og preferanser – og på sikt bygge et anbefalingssystem – er det viktig å forstå kanselleringsmønstrene til kundene.

Derfor kan vi velge å beholde kansellerte transaksjonene i datasettet, men markere de slik at de kan analyseres separat senere.  

Denne tilnærmingen gir to fordeler:  

1. **Bedre klustring:**  
   Kanselleringsmønstre kan reflektere spesielle typer kundeadferd eller preferanser, og bør derfor inngå i analysen.

2. **Mer presise anbefalinger:**  
   Ved å ta hensyn til hvilke produkter som ofte kanselleres, kan anbefalingssystemet unngå å foreslå produkter med høy sannsynlighet for retur eller kansellering. Det kan forbedrer kvaliteten på anbefalingene.  


In [ ]:
# Prosentandelen kansellerte transaksjoner
cancelled_percentage = (cancelled_transactions.shape[0] / df.shape[0]) * 100

print(f"Andelen kansellerte transaksjoner i datasettet er: {cancelled_percentage:.2f}%")


### 3.4 | Rette opp rariteter

Vi finner unike produktkoder (*StockCode*) i datasettet, og visualisere de 10 mest brukte sammen med andelen av datasettet de utgjør.

Vi undersøker om alle produktkodene i `StockCode` er gyldige.  

1. Hvor mange talltegn (0–9) inneholder hver unike `StockCode`.  
2. Hvor mange koder har ingen eller ett talltegn?  
3. Hvilke kodene avviker fra normalen?  
4. Hvor stor prosentandel av datasettet utgjør disse kodene?  
5. Bør vi beholde eller fjerne disse kodene fra analysen?

In [ ]:
# Lager liste over unike StockCodes
unique_stock_codes = df['StockCode'].unique()

# Antall talltegn i hver kode
numeric_char_counts_in_unique_codes = (
    pd.Series(unique_stock_codes)
    .apply(lambda x: sum(c.isdigit() for c in str(x)))
    .value_counts()
)

# Lager fordeling (value_counts) av antall talltegn per kode
print("Antall koder per antall talltegn:")
print(numeric_char_counts_in_unique_codes)


# Finner koder med 0 eller 1 talltegn
anomalous_stock_codes = [
    code for code in unique_stock_codes
    if sum(c.isdigit() for c in str(code)) in (0, 1)
]
print("\nUnormale StockCodes:")
for code in anomalous_stock_codes:
    print(code)

# Prosentandelen av rader i df med unormale koder
percentage_anomalous = (df['StockCode'].isin(anomalous_stock_codes).sum() / len(df)) * 100
print(f"\nAndel av datasettet med unormale/anomalier: {percentage_anomalous:.2f}%")

# Fjerner disse
df = df[~df['StockCode'].isin(anomalous_stock_codes)]


### 3.5 | Renser beskrivelseskolonnen

Vi finner antall forekomster per unik *Description*, og plotter de 30 mest bruke. For å få et bilde av hva som dominerer i datasettet.

In [ ]:
# Finn topp 30 mest brukte beskrivelser
top_30 = df['Description'].value_counts().head(30)

# Plot
top_30.plot(kind="barh", figsize=(10,6), color="#ff6200")
plt.title("Topp 30 mest brukte beskrivelser")
plt.show()

In [ ]:
# Finner beskrivelser som inneholder små bokstaver
lowercase_desc = [d for d in df['Description'].dropna().unique() if any(c.islower() for c in str(d))]
lowercase_desc[:25]  # vis de første 25

Hvilke av er produkter og hvilke er potensielt tjenester eller metadata?

In [ ]:
# Fjern rader med service-relaterte beskrivelser
service_related = ["Next Day Carriage", "High Resolution Image"]
df = df[~df['Description'].isin(service_related)]

# Standardiser resterende til uppercase
df['Description'] = df['Description'].astype(str).str.upper()

### 3.6 | Behandler nullverdier i UnitPrice
Undersøker `UnitPrice` for å se etter mistenkelige verdier.

1. Bruk `.describe()` på `UnitPrice`.  
2. Hva er minimumsverdien? Hva forteller det oss?  
3. Hvor mange rader har `UnitPrice = 0`?  
4. Diskuter: Hva bør vi gjøre med disse radene?  

In [ ]:
df = df[df['UnitPrice'] > 0]


### 3.7 | Uteliggere?
Klustringalgoritmer kan være følsommer når det kommer til både skala og uteliggere (outliers). Uteliggere kan trekke centroidene ut av posisjon og derfor gi dårlige klustere.

MEN dataene er fortsatt transaksjonsdata. Om vi fjerner uteliggere nå, kan vi risikere å kaste bort info som kan være nyttig senere når vi skal lage variabler knyttet til kjøpsmønsteret.

Vi resetter indexen, og sier oss ferdig med første del av databehandlingen.

In [ ]:
# Resette indeksen etter all rensing
df.reset_index(drop=True, inplace=True)

# Antall rader i det rensede datasettet
print(f"Antall rader i datasettet etter rensing: {df.shape[0]}")

# Oppsummering – Datahåndtering

Vi har renset og bearbeidet datasettet slik at det er klart for videre analyse:

- ✅ Håndtert manglende verdier:
  - Fjernet rader uten `CustomerID` eller `Description`.

- ✅ Håndtert duplikater:
  - Fjernet helt identiske rader.

- ✅ Behandlet kansellerte transaksjoner:
  - Merket rader som `Cancelled` eller `Completed` for senere analyse.

- ✅ Ryddet opp i StockCode:
  - Identifisert og fjernet anomalier (f.eks. `POST`, `BANK CHARGES`).

- ✅ Renset Description:
  - Fjernet tjeneste-relaterte beskrivelser (`Next Day Carriage` og `High Resolution Image`).
  - Standardisert resterende beskrivelser til UPPERCASE.

- ✅ Behandlet nullverdier i UnitPrice:
  - Fjernet rader med `UnitPrice = 0`.

- ✅ Uteliggerbehandling:
  - Utsetter dette til etter feature engineering (kundeprofiler).

**Datasettet er klart til neste fase - som handler om å bygge:**
Der skal vi lage kundesentrerte variabler og segmentere kundene med clustering.